[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/gdslab/d2spy/blob/main/docs/guides/notebooks/01_creating_workspace_data.ipynb)

# Creating new projects, flights, and data products
*This guide will walk you through the steps for creating new data on a Data to Science (D2S) instance.*

To get started, you will need to import the `Auth` and `Workspace` modules.

In [ ]:
# Uncomment and run the following line if working out of Google Colab
# !pip install d2spy

In [ ]:
from datetime import date

from d2spy.auth import Auth
from d2spy.workspace import Workspace

You must login to a D2S instance before you can request any data. The `Auth` module will be used to handle authenticating with D2S and requesting an authorization token. You will need to provide the module with the URL to your D2S instance, the email address associated with your D2S account, and your password when prompted.

In [ ]:
# URL to a D2S instance
d2s_url = "http://localhost:8000"

# Login with your email address
auth = Auth(d2s_url)
user = auth.login(email="yourD2Semail@example.com")

After successfully logging in to a D2S instance, you will need to create a new workspace using the user session created in the previous step.

In [ ]:
# Workspace session
workspace = Workspace(d2s_url, auth.session)

## Creating a new project

To create a new project in our workspace, we can use the Workspace module's [`add_project`](https://py.d2s.org/workspace/#d2spy.workspace.Workspace.add_project) method as follows:

In [ ]:
# At minimum, must provide title, description, and location (dict in GeoJSON Feature format)
project = workspace.add_project(
    title="Project title",
    description="Project for testing d2spy package.",
    location={
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [
                [
                  [-86.944981783977838, 41.444435853085622],
                  [-86.943319754949272, 41.444435046238446],
                  [-86.94332056379109, 41.443505552529658],
                  [-86.944982569102066, 41.443506359350643],
                  [-86.944981783977838, 41.444435853085622]
                ]
            ]
        },
        "properties": {
            "prop0": "value0"
        }
    }
)
print(project)

The Workspace module's `add_project` returns a [`Project`](https://py.d2s.org/project/) instance we can use to create new flights in the project, retrieve existing flights, and update the initially provided project details.

For example, we can use the Project [`update`](https://py.d2s.org/project/#d2spy.models.project.Project.update) method to change the project title.

In [ ]:
print(f"Original title: {project.title}")
project.update(title="My First D2S Project")
print(f"Updated title: {project.title}")

## Adding a flight to a project

Since this is a new project, we do not have any available flights yet. We can add a flight using the Project module's [`add_flight`](https://py.d2s.org/project/#d2spy.models.project.Project.add_flight) method.

**Important**

The `sensor` and `platform` attributes have some limits on what values will be accepted. `Sensor` will only accept the values: "RGB", "Multispectral", "LiDAR", and "Other". `Platform` expects the values "Phantom_4", "M300", or "M350". It can be provided with an alternative value if none of these options are suitable. 

Refer to the [documentation](https://py.d2s.org/project/#d2spy.models.project.Project.add_flight) for more details.

In [ ]:
# Example flight form
flight = project.add_flight(
    acquisition_date=date(2023, 6, 3),  # string in YYYY-MM-DD format also acceptable
    altitude=40,
    side_overlap=85,
    forward_overlap=85,
    sensor="RGB",
    platform="M350"
)
print(flight)

## Uploading a data product to a flight

Now that a flight has been created, you can start uploading data products to the flight. Note, the data product will not be immediately available after the upload completes. When D2S receives a new GeoTIFF, it will check if it is in a cloud-optimized format, and if not, perform this conversion. This process may take several minutes to complete.

The Flight module's [`add_data_product`](https://py.d2s.org/flight/#d2spy.models.flight.Flight.add_data_product) method will be used to upload a GeoTIFF located on the local machine. You will need to have a GeoTIFF and know its filepath for this next cell.

**Important**

The `data_type` has some limits on what values will be accepted. `data_type` expects the values "dsm", "point_cloud", or "ortho". It can be provided with an alternative value if none of these options are suitable. 

Refer to the [documentation](https://py.d2s.org/flight/#d2spy.models.flight.Flight.add_data_product) for more details.

In [ ]:
my_geotiff = "/full/path/to/my/data_product.tif"
flight.add_data_product(
    filepath=my_geotiff,
    data_type="dsm"
)

When D2S has finished processing the upload the Flight module's [`get_data_products`](https://py.d2s.org/flight/#d2spy.models.flight.Flight.get_data_products) method can be used to find the uploaded data product and its static URL.

In [ ]:
data_products = flight.get_data_products()
try:
    print(data_products[0].url)
except IndexError:
    print("Data product still processing or an unexpected error has occurred")

These are all the methods you need to know for creating data on a D2S instance. Once finished adding your data, you can revoke your authorization session with the Auth logout method.

In [ ]:
# Removes access token from future requests
auth.logout()